# Ablation: Semi-Supervised Architope - for Reviews
---
- This code Implements Algorithm 3.2 of the "PC-NNs" paper.

#### Mode: Code-Testin Parameter(s)

In [26]:
trial_run = True

### Meta-parameters
In Grid... .py file

### Hyperparameters

Only turn of if running code directly here, typically this script should be run be called by other notebooks.  

In [27]:
# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"

### Import

In [ ]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Pre-process Data
if Option_Function != "Motivational_Example": 
    exec(open('Financial_Data_Preprocessor.py').read())
else:
    if Option_Function == 'California_Housing':
        exec(open('Prepare_Data_California_Housing.py').read())
    else:
        print(1)
        exec(open('Motivational_Example.py').read())
        print("Training Data size: ",X_train.shape[0])
# Import time separately
import time
### Set Seed
random.seed(2021)
tf.random.set_seed(2021)

Deep Feature Builder - Ready
Deep Classifier - Ready


In [ ]:
Option_Function = 'Sythetic'
if Option_Function == 'Sythetic':
    # Generate Data
    ## Input Data
    N_train = int(np.round(N*test_size_ratio))
    N_test = int(np.round(N*(1-test_size_ratio)))
    X_train = np.random.uniform(low=0,high=1,size=np.array([N_train,D_in]))
    X_test = np.random.uniform(low=0,high=1,size=np.array([N_test,D_in]))
    ## Get Outputs
    random_projector = np.random.normal(size = [D_in,1])
    y_train = np.matmul(X_train,random_projector).reshape(-1,)
    y_test = np.matmul(X_test,random_projector).reshape(-1,)
    y_train = np.array([f_unknown(xi) for xi in y_train])
    y_test = np.array([f_unknown(xi) for xi in y_test])
    # Add Noise to Training Data
    training_noise = np.sqrt(np.abs(noise_level))*np.random.standard_t(df = tailedness, size=y_train.shape)
    y_train += training_noise
    
    if D_in ==1 :
        # Visualize Data
        sns.set()
        # Initialize Plot #
        #-----------------#
        plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
        # Format Plot #
        #-------------#
        plt.title("Signal and Observations")
        plt.xlabel("x")
        plt.ylabel("y")

        # Generate Plots #
        #----------------#
        # Plot Signal
        plt.scatter(X_train,
                    y_train,
                    label ='f(x)',
                    color='blue',
                    s = 1.5)
        plt.scatter(X_test,
                    y_test,
                    label ='y',
                    color='red',
                    s = 3)
        plt.legend(loc="upper left")


        # Export #
        #--------#
        # SAVE Figure to .eps
        plt.savefig('./outputs/plotsANDfigures/Signal_Synthetic'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
        plt.show()

        # Coercsion
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)

#### Pre-Process:
- Convert Categorical Variables to Dummies
- Remove Bad Column
- Perform Training/Test Split

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Standardize Data
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))
sns.set()
plt.show()

# Random Lipschitz Partition Builder

We implement the random paritioning method of [Yair Bartal](https://scholar.google.com/citations?user=eCXP24kAAAAJ&hl=en):
- [On approximating arbitrary metrices by tree metrics](https://dl.acm.org/doi/10.1145/276698.276725)

The algorithm is summarized as follow:

---

## Algorithm:
 1. Sample $\alpha \in [4^{-1},2^{-1}]$ randomly and uniformly,
 2. Apply a random suffle of the data, (a random bijection $\pi:\{i\}_{i=1}^X \rightarrow \mathbb{X}$),
 3. For $i = 1,\dots,I$:
   - Set $K_i\triangleq B\left(\pi(i),\alpha \Delta \right) - \bigcup_{j=1}^{i-1} P_j$
 
 4. Remove empty members of $\left\{K_i\right\}_{i=1}^X$.  
 
 **Return**: $\left\{K_i\right\}_{i=1}^{\tilde{X}}$.  
 
 For more details on the random-Lipschitz partition of Yair Bartal, see this [well-written blog post](https://nickhar.wordpress.com/2012/03/26/lecture-22-random-partitions-of-metric-spaces/).

## Define Random Partition Builder

#### Explicit Partion Builder:
Implements exactly Algorithm 2:

In [ ]:
def Random_Lipschitz_Partioner(X_in,
                               y_in,
                               N_parts_to_get=4):

    # Compute Size of each part
    size_part_reference = int(round(X_in.shape[0]/N_parts_to_get))

    # Apply random bijection #
    #------------------------#
    ## Get random bijection indices
    random_bijection_indices = np.random.choice(range(X_in.shape[0]),size=X_in.shape[0], replace=False)
    ## Apply random bijections
    X_in_shuffled = X_in[random_bijection_indices,:]
    y_in_shuffled = y_in[random_bijection_indices,:]

    # Initialize Lists #
    #------------------#
    X_parts = []
    y_parts = []

    for i_th_part_to_get in range(N_parts_to_get):
        # Build random balls #
        #--------------------#
        ## Sample random radius
        size_part = int(np.maximum(1,np.round(size_part_reference*np.random.uniform(low=.5,high=1.5,size=1)[0])))
        ## Sample random point
        X_center_loop_index = np.random.choice(range(X_in_shuffled.shape[0]),size=1, replace=False)
        X_center_loop = X_in_shuffled[X_center_loop_index,:]
        ## Compute Typical Distances from Center
        distances_loop = X_center_loop-X_in_shuffled
        distances_loop = np.linalg.norm(distances_loop, axis=1)

        # Remove Random Ball from Dataset
        if size_part <= len(distances_loop):
            ## Identify indices
            indices_smallest_to_random_ball = np.argsort(distances_loop)[:size_part]
        else:
            print('Final Loop')
            indices_smallest_to_random_ball = np.array(range(X_in_shuffled.shape[0]))
        ## Extract Parts
        X_current_part_loop = X_in_shuffled[indices_smallest_to_random_ball,:]
        y_current_part_loop = y_in_shuffled[indices_smallest_to_random_ball,:]
        ## Append to List of Parts
        X_parts.append(X_current_part_loop)
        y_parts.append(y_current_part_loop)

        # Remove Selected Entries From Array #
        #------------------------------------#
        X_in_shuffled = np.delete(X_in_shuffled,indices_smallest_to_random_ball,axis=0)
        y_in_shuffled = np.delete(y_in_shuffled,indices_smallest_to_random_ball,axis=0)

        # Failsafe if procedure has terminated
        if X_in_shuffled.shape[0] == 0:
            print('breaking early')
            break
    # Count Number of Parts Generated        
    N_parts_generated = len(X_parts)
    # Output Parts
    return X_parts, y_parts, N_parts_generated

# Ablation Function

In [ ]:
def get_PCNNs(N_parts,
              X_train,
              y_train,
              X_test,
              y_test):

    # Initialization(s) #
    #-------------------#
    N_neurons = 0
    L_timer = 0
    P_timer = 0
    Mean_Width_Subnetworks = 0

    # Partitioner Begin #
    #-------------------#
    import time
    partitioning_time_begin = time.time()
    print('-------------------------------------------------------')
    print('Randomly Initialized Parts - Via Randomized Algorithm 2')
    print('-------------------------------------------------------')
    if Partition_using_Inputs == True:
        X_parts_list, y_parts_list, N_parts_Generated_by_Algo_2 = Random_Lipschitz_Partioner(X_train.to_numpy(),
                                                                                             y_train.reshape(-1,1),
                                                                                             N_parts)
    else:
        X_parts_list, y_parts_list, N_parts_Generated_by_Algo_2 = Random_Lipschitz_Partioner(y_train.reshape(-1,1),
                                                                                             X_train.to_numpy(),
                                                                                             N_parts)
    partitioning_time = time.time() - partitioning_time_begin
    print('The_parts_listhe number of parts are: ' + str(N_parts_Generated_by_Algo_2)+'.')
    ############# Partitioner End ########

    print('-----------------------------------------------------')
    print('Training Sub-Networks on Each Randomly Generated Part')
    print('-----------------------------------------------------')
    # Time-Elapse (Start) for Training on Each Part #
    PCNN_timer = time.time(); PCNN_timer = -math.inf; N_params_Architope = 0; N_params_tally = 0
    # Remove Eager Execution Error(s)
    tf.compat.v1.disable_eager_execution()
    # Automatically Initialize Correct Input/Output Dimension(s)
    param_grid_Vanilla_Nets['input_dim'] = [X_train.shape[1]]; param_grid_Vanilla_Nets['output_dim'] = [1]
    param_grid_Deep_Classifier['input_dim'] = [X_train.shape[1]]
    # Decide if/or not to tie neuron numbers of sub-patterns together
    if Tied_Neurons_Q == True:
        param_grid_Vanilla_Nets['height'] = [int(np.maximum(round(param_grid_Vanilla_Nets['height'][0]/N_parts),min_width))]
        param_grid_Vanilla_Nets['epochs'] = [int(np.maximum(round(param_grid_Vanilla_Nets['epochs'][0]/int(round(np.sqrt(N_parts)))),min_epochs))]
#         param_grid_Deep_Classifier['height'] = [int(np.maximum(round(param_grid_Deep_Classifier['height'][0]/N_parts),min_width))]


    for current_part in range(N_parts_Generated_by_Algo_2):
        # Update User #
        #-------------#
        print('-----------------------------------------------------------')
        print('Currently Training Part: '+str(current_part)+'/'+str(N_parts_Generated_by_Algo_2 )+'Total Parts.')
        print('-----------------------------------------------------------')

        # Timer for Part
        part_training_timer = time.time()
        # Get Data for Sub-Pattern
        X_loop = pd.DataFrame(X_parts_list[current_part])
        y_loop = (y_parts_list[current_part]).reshape(-1,)
        # Train ffNN
        if randomize_subpattern_construction == False:
            y_hat_part_loop, y_hat_part_loop_test, N_neurons_PCNN_loop = build_ffNN(n_folds = 4, 
                                                                                  n_jobs = n_jobs,
                                                                                  n_iter = n_iter, 
                                                                                  param_grid_in = param_grid_Vanilla_Nets, 
                                                                                  X_train= X_loop, 
                                                                                  y_train=y_loop,
                                                                                  X_test_partial=X_train,
                                                                                  X_test=X_test,
                                                                                  NOCV=True)
        else:
            y_hat_part_loop, y_hat_part_loop_test, N_neurons_PCNN_loop = build_ffNN_random(X_loop,
                                                                                           X_train,
                                                                                           X_test,
                                                                                           y_loop,
                                                                                           param_grid_Vanilla_Nets)
        # Reshape y
        ## Training
        y_train.shape = (y_train.shape[0], param_grid_Vanilla_Nets['output_dim'][0])
        y_hat_part_loop.shape = (y_hat_part_loop.shape[0], param_grid_Vanilla_Nets['output_dim'][0])
        ## Testing
        y_test.shape = (y_test.shape[0], param_grid_Vanilla_Nets['output_dim'][0])
        y_hat_part_loop_test.shape = (y_hat_part_loop_test.shape[0], param_grid_Vanilla_Nets['output_dim'][0])

        # Append predictions to data-frames
        ## If first prediction we initialize data-frames
        if current_part==0:
            # Register quality
            training_quality = np.array(np.abs(y_hat_part_loop-y_train)).reshape(y_hat_part_loop.shape[0],1)
            # Save Predictions
            predictions_train = y_hat_part_loop.reshape(y_hat_part_loop.shape[0],1)
            predictions_test = y_hat_part_loop_test.reshape(y_hat_part_loop_test.shape[0],1)


        ## If not first prediction we append to already initialized dataframes
        else:
        # Register Best Scores
            #----------------------#
            # Write Predictions 
            # Save Predictions
            y_hat_train_loop = y_hat_part_loop.reshape(predictions_train.shape[0],1)
            predictions_train = np.append(predictions_train,y_hat_train_loop,axis=1)
            y_hat_test_loop = y_hat_part_loop_test.reshape(predictions_test.shape[0],1)
            predictions_test = np.append(predictions_test,y_hat_test_loop,axis=1)

            # Evaluate Errors #
            #-----------------#
            # Training
            prediction_errors = np.abs(y_hat_train_loop-y_train)
#             prediction_errors = np.abs(y_hat_train_loop-y_loop)
            training_quality = np.append(training_quality,prediction_errors.reshape(training_quality.shape[0],1),axis=1)
        #==============================#
        # Update Performance Metric(s) #
        #==============================#
        part_training_timer = time.time() - part_training_timer
        # L-Time
        L_timer += partitioning_time
        # P-Time
        P_timer = max(P_timer,part_training_timer)
        # N. Params
        N_neurons += N_neurons_PCNN_loop
        # Mean Width for Sub-Network(s)
        Mean_Width_Subnetworks += param_grid_Vanilla_Nets['height'][0]

    # Take Mean of Width(s)
    Mean_Width_Subnetworks = Mean_Width_Subnetworks/N_parts_Generated_by_Algo_2
    print('-----------------------')
    print('Training Deep Zero-Sets')
    print('-----------------------')


    # Time Elapsed for Training Deep Zero-Sets
    Deep_Zero_Sets_timer = time.time()

    ## Initialize Classes Labels
    if softmax_layer == False:
        # No pooling (classical)
        partition_labels_training_integers = np.argmin(training_quality,axis=-1)
    else:
        # Max Pooling
#         partition_labels_training_integers = (training_quality == training_quality.min(axis=1)[:,None]).astype(int)
        partition_labels_training_integers = np.apply_along_axis(softminn, 1, training_quality).astype(int)
    partition_labels_training = pd.DataFrame(pd.DataFrame(partition_labels_training_integers) == 0)
    ## Build Classes
    for part_column_i in range(1,(training_quality.shape[1])):
        partition_labels_training = pd.concat([partition_labels_training,
                                               (pd.DataFrame(partition_labels_training_integers) == part_column_i)
                                              ],axis=1)
    ## Convert to integers
    partition_labels_training = partition_labels_training+0
    ## Train simple deep classifier
    tf.compat.v1.disable_eager_execution()
    exec(open('Grid_Enhanced_Network.py').read())
    if randomize_subpattern_construction_Deep_ZeroSets == False:
#         print(partition_labels_training)
        param_grid_Deep_Classifier['epochs'] = [int(np.maximum(round(param_grid_Deep_Classifier['epochs'][0]),min_epochs_classifier))]
#         param_grid_Deep_Classifier['height'] = param_grid_Deep_Classifier['height']
#         print(param_grid_Deep_Classifier)
        predicted_classes_train, predicted_classes_test, N_params_deep_classifier = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                                                            n_jobs = n_jobs, 
                                                                                                            n_iter =n_iter, 
                                                                                                            param_grid_in = param_grid_Deep_Classifier, 
                                                                                                            X_train = X_train.values, 
                                                                                                            y_train = partition_labels_training.values,
                                                                                                            X_test = X_test.values)
        # Get Binary Classes (Discontinuous Unit)
        ## Training Set
        predicted_classes_train = ((predicted_classes_train>gamma)*1).astype(int)
        ## Testing Set
        predicted_classes_test = ((predicted_classes_test > gamma)*1).astype(int)
        # Get PC-NN Prediction(s)
        ## Train
        PCNN_prediction_y_train = (predictions_train*predicted_classes_train).sum(axis=1)
        ## Test
        PCNN_prediction_y_test = (predictions_test*predicted_classes_test).sum(axis=1)
    else:
        if N_parts > 1:
            print(partition_labels_training)
            partition_labels_training_dzs = np.argmin(partition_labels_training.to_numpy(),axis=-1)
            for j in range(10):
                print('---')
            print(partition_labels_training_dzs)
            PCNN_prediction_y_train, PCNN_prediction_y_test, N_params_deep_classifier = build_deep_classifier_random(X_train_in = X_train,
                                                                                                                     X_train_in_full = X_train,
                                                                                                                     X_test_in = X_test,
                                                                                                                     predictions_test_in = predictions_test,
                                                                                                                     predictions_train_in = predictions_train,
                                                                                                                     classes_in = partition_labels_training_dzs,
                                                                                                                     param_grid_in = param_grid_Deep_Classifier)
        else:
            print('Entering N Parts == 1 case!')
            PCNN_prediction_y_train = predictions_train.reshape(-1,param_grid_Vanilla_Nets['output_dim'][0])
            PCNN_prediction_y_test = predictions_test.reshape(-1,param_grid_Vanilla_Nets['output_dim'][0])
            N_params_deep_classifier = 0
    # End Timer
    Deep_Zero_Sets_timer = time.time() - Deep_Zero_Sets_timer

    print('-----------------------------------')
    print('Computing Final Performance Metrics')
    print('-----------------------------------')
    # Time-Elapsed Training Deep Classifier

    # Update Times
    L_timer +=Deep_Zero_Sets_timer
    P_timer +=Deep_Zero_Sets_timer
    # Update Number of Neurons Used
    N_neurons_subPatterns = N_neurons
    N_neurons_deep_Zero_Sets = (param_grid_Deep_Classifier['height'][0])*(param_grid_Deep_Classifier['depth'][0])
    N_neurons = N_neurons_deep_Zero_Sets + N_neurons_subPatterns



    # Compute Peformance
    performance_PCNN = reporter(y_train_hat_in=PCNN_prediction_y_train,y_test_hat_in=PCNN_prediction_y_test,
                                y_train_in=y_train,
                                y_test_in=y_test)
    # Write Performance
    performance_PCNN.to_latex((results_tables_path+"PCNN_full_performance.tex"))

    # Update User
    print(performance_PCNN)

    ### Model Complexity/Efficiency Metrics
    # Build AIC-like Metric #
    #-----------------------#
    AIC_like = 2*(N_neurons - np.log((performance_PCNN['test']['MAE'])))
    AIC_like = np.round(AIC_like,3)
    Efficiency = np.log(N_neurons) *(performance_PCNN['test']['MAE'])
    Efficiency = np.round(Efficiency,3)


    # Build Table #
    #-------------#
    PCNN_Model_Complexity = pd.DataFrame({'L-time': [L_timer],
                                               'P-time':[P_timer],
                                               'N_params_expt': [N_neurons],
                                               'AIC-like': [AIC_like],
                                               'Eff': [Efficiency],
                                               'N. Parts':[N_parts_Generated_by_Algo_2]})


    # Write Required Training Time(s)
    PCNN_Model_Complexity.to_latex((results_tables_path+"PCNN_full_model_complexities.tex"))

    #--------------======---------------#
    # Display Required Training Time(s) #
    #--------------======---------------#
    print(PCNN_Model_Complexity)
    
    
    
    #########################################
    for j in range(10):
        print('#------------------------------#')
    #########################################
    print('# ---- Getting Benchmarks ---- #')
    #########################################
    for j in range(10):
        print('#------------------------------#')
    #########################################
    print('Training PCNN-lgt')
    # Time-Elapsed Training linear classifier
    Architope_logistic_classifier_training_begin = time.time()
    if N_parts > 1:
        if X_train.shape[0]>(5*N_parts): # Failsafe against small CV set
            parameters = {'penalty': ['none'], 'C': [0.1]}
            lr = LogisticRegression(random_state=2020)
            cv = RepeatedStratifiedKFold(n_splits=2, 
                                         n_repeats=n_iter, 
                                         random_state=0)
            classifier = RandomizedSearchCV(lr, 
                                            parameters, 
                                            random_state=2020)
        else:
            classifier = LogisticRegression(random_state=2020)

        # Initialize Classes Labels
        partition_labels_training = np.argmin(training_quality,axis=-1)
        # Train Logistic Classifier #
        #---------------------------#
        # Supress warnings caused by "ignoring C" for 'none' penalty and similar obvious warnings
        warnings.simplefilter("ignore")
        # Train Classifier
        classifier.fit(X_train, partition_labels_training)
    if N_parts >1 :
        #### Write Predicted Class(es)
        # Training Set
        if X_train.shape[0]>(5*N_parts): # Failsafe against small CV set
            predicted_classes_train_logistic_BM = classifier.best_estimator_.predict(X_train)
        else:
            predicted_classes_train_logistic_BM = classifier.predict(X_train)
        Architope_prediction_y_train_logistic_BM = np.take_along_axis(predictions_train, 
                                                                      predicted_classes_train_logistic_BM[:,None], 
                                                                      axis=1)
        # Testing Set
        if X_train.shape[0]>(5*N_parts): # Failsafe against small CV set
            predicted_classes_test_logistic_BM = classifier.best_estimator_.predict(X_test)
        else:
            predicted_classes_test_logistic_BM = classifier.predict(X_test)
        Architope_prediction_y_test_logistic_BM = np.take_along_axis(predictions_test, 
                                                                     predicted_classes_test_logistic_BM[:,None], 
                                                                     axis=1)
    else:
        #### Write Predicted Class(es)
        # Training Set
        Architope_prediction_y_train_logistic_BM = predictions_train
        # Testing Set
        Architope_prediction_y_test_logistic_BM = predictions_test    
    # Extract Number of Parameters Logistic Regressor
    if N_parts > 1:
        if X_train.shape[0]>(5*N_parts): # Failsafe against small CV set
            N_params_best_logistic = (classifier.best_estimator_.coef_.shape[0])*(classifier.best_estimator_.coef_.shape[1]) + len(classifier.best_estimator_.intercept_)
        else:
            N_params_best_logistic = (classifier.coef_.shape[0])*(classifier.coef_.shape[1]) + len(classifier.intercept_)
    else:
        N_params_best_logistic = 1
    N_params_best_logistic = N_params_best_logistic + N_neurons_subPatterns*N_parts    
    # Time-Elapsed Training linear classifier
    Architope_logistic_classifier_training = time.time() - Architope_logistic_classifier_training_begin
    #### Compute Performance
    # Compute Peformance
    performance_architope_ffNN_logistic = reporter(y_train_hat_in=Architope_prediction_y_train_logistic_BM,
                                        y_test_hat_in=Architope_prediction_y_test_logistic_BM,
                                        y_train_in=y_train,
                                        y_test_in=y_test)
    # Write Performance
    performance_architope_ffNN_logistic.to_latex((results_tables_path+"Architopes_logistic_performance.tex"))
    
    ##### --- #####
    print('Training PCNN-Bagged')
    ##### --- #####
    # Time for Bagging
    Bagging_ffNN_bagging_time_begin = time.time()
    # Train Bagging Weights in-sample
    bagging_coefficients = LinearRegression().fit(predictions_train,y_train)
    # Predict Bagging Weights out-of-sample
    bagged_prediction_train = bagging_coefficients.predict(predictions_train)
    bagged_prediction_test = bagging_coefficients.predict(predictions_test)
    # Write number of trainable bagging parameters
    N_bagged_parameters = len(bagging_coefficients.coef_) + 1
    # Time for Bagging
    Bagging_ffNN_bagging_time = time.time() - Bagging_ffNN_bagging_time_begin
    # Compute Peformance
    performance_bagged_ffNN = reporter(y_train_hat_in=bagged_prediction_train,
                                        y_test_hat_in=bagged_prediction_test,
                                        y_train_in=y_train,
                                        y_test_in=y_test)
    # Write Performance
    performance_bagged_ffNN.to_latex((results_tables_path+"ffNN_Bagged.tex"))
    
    for jj in range(5):
        print('-----------------------')
    print('...Returning Results...')
    for jj in range(5):
        print('-----------------------')
    # Return Output(s)
    return performance_PCNN, PCNN_Model_Complexity, N_parts_Generated_by_Algo_2, N_neurons, N_neurons_subPatterns,N_neurons_deep_Zero_Sets, Mean_Width_Subnetworks, performance_architope_ffNN_logistic, N_params_best_logistic, performance_bagged_ffNN, Bagging_ffNN_bagging_time, Architope_logistic_classifier_training, Deep_Zero_Sets_timer

---
# Perform Ablation:
---

In [ ]:
# Reload
exec(open('Helper_Functions.py').read())
exec(open('Grid_Enhanced_Network.py').read())
# Initialize 
if 'N_parts_possibilities' not in locals(): #If no custom list of parts to check is provided:
    N_parts_possibilities = np.unique(np.round(np.linspace(N_min_parts,N_max_plots,num=N_plot_finess))).astype(int)

# Get Performance Metric
for inplicit_N_parts_loop in range(len(N_parts_possibilities)):
    ### UPDATE USER ###
    for k in range(10):
        print('--------------------------------------')
    print('Ablation Completion Percentage:',(inplicit_N_parts_loop/N_plot_finess))
    for k in range(10):
        print('--------------------------------------')
    
    # Implicitly Set: Current Number of Parts
#     q_implicit_N_parts_loop = q_implicit_N_parts_possibilities[inplicit_N_parts_loop]
    N_parts_possibilities_loop = N_parts_possibilities[inplicit_N_parts_loop]
    # Run Algos. 1+2
    performance_Architope_loop, Architope_Model_Complexity_full_loop, N_parts_Generated_by_Algo_2_loop, N_params_architope_loop, N_neurons_subPatterns_loop, N_neurons_deep_Zero_Sets_loop, height_mean_loop, performance_PCNN_ffNN_logistic_loop, N_params_PCNN_logistic_loop,performance_bagged_ffNN_loop, baggin_time_loop, logistic_time_loop, Deep_Zero_Sets_timer_loop = get_PCNNs(N_parts_possibilities_loop,X_train,y_train,X_test,y_test)
    # Reshape
    performance_Architope_loop = performance_Architope_loop.to_numpy().reshape([3,2,1])
    Architope_Model_Complexity_full_loop = Architope_Model_Complexity_full_loop.to_numpy().reshape([1,6,1])
    performance_PCNN_ffNN_logistic_loop = performance_PCNN_ffNN_logistic_loop.to_numpy().reshape([3,2,1])
    performance_bagged_ffNN_loop = performance_bagged_ffNN_loop.to_numpy().reshape([3,2,1])
    # Record
    if inplicit_N_parts_loop == 0:
        # Don't count partitioner if only one parts is active!
        if N_parts_possibilities_loop <= 1:
            Architope_Model_Complexity_full_loop[:,1] = Architope_Model_Complexity_full_loop[:,0]
            N_neurons_deep_Zero_Sets_loop = 0
        # Record Model Complexities Otherwise    
        performance_Architope_history = performance_Architope_loop
        Architope_Model_Complexity_history = Architope_Model_Complexity_full_loop
        N_parts_Generated_by_Algo_2_history = N_parts_Generated_by_Algo_2_loop
        N_params_subPatterns_hist = N_neurons_subPatterns_loop
        N_neurons_deep_Zero_Sets_hist = N_neurons_deep_Zero_Sets_loop
        N_params_architope_hist = N_neurons_deep_Zero_Sets_loop + N_neurons_subPatterns_loop
        height_mean_hist = height_mean_loop
        N_neurons_per_input = N_neurons_deep_Zero_Sets_loop + int(round(N_neurons_subPatterns_loop/N_parts_possibilities_loop))
        ### BENCHMARKs
        ### Logistic PCNN
        performance_PCNN_ffNN_logistic_hist = performance_PCNN_ffNN_logistic_loop
        N_params_PCNN_logistic_hist = N_params_PCNN_logistic_loop
        logistic_time_hist =  logistic_time_loop
        baggin_time_hist = baggin_time_loop
        ### Bagged PCNNs
        performance_bagged_ffNN_hist = performance_bagged_ffNN_loop
        ### Misc
        Deep_Zero_Sets_timer_hist = Deep_Zero_Sets_timer_loop
    else:
        performance_Architope_history = np.concatenate((performance_Architope_history,performance_Architope_loop),axis=2)
        Architope_Model_Complexity_history = np.concatenate((Architope_Model_Complexity_history,Architope_Model_Complexity_full_loop),axis=2)
        N_parts_Generated_by_Algo_2_history = np.append(N_parts_Generated_by_Algo_2_history,N_parts_Generated_by_Algo_2_loop)
        N_params_architope_hist = np.append(N_params_architope_hist,N_params_architope_loop)
        N_params_subPatterns_hist = np.append(N_params_subPatterns_hist,N_neurons_subPatterns_loop)
        N_neurons_deep_Zero_Sets_hist = np.append(N_neurons_deep_Zero_Sets_hist,N_neurons_deep_Zero_Sets_loop)
        height_mean_hist = np.append(height_mean_hist,height_mean_loop)
        N_neurons_per_input = np.append(N_neurons_per_input,(N_neurons_deep_Zero_Sets_loop + int(round(N_neurons_subPatterns_loop/N_parts_possibilities_loop))))
        ### Logistic PCNN
        performance_PCNN_ffNN_logistic_hist = np.concatenate((performance_PCNN_ffNN_logistic_hist,
                                                              performance_PCNN_ffNN_logistic_loop),
                                                             axis=2)
        N_params_PCNN_logistic_hist = np.append(N_params_PCNN_logistic_hist,N_params_PCNN_logistic_loop)
        logistic_time_hist = np.append(logistic_time_hist,logistic_time_loop)
        ### Bagged Performance
        performance_bagged_ffNN_hist = np.concatenate((performance_bagged_ffNN_hist,
                                                       performance_bagged_ffNN_loop),
                                                      axis=2)
        baggin_time_hist = np.append(baggin_time_hist,baggin_time_loop)
        ### Misc
        Deep_Zero_Sets_timer_hist = np.append(Deep_Zero_Sets_timer_hist,Deep_Zero_Sets_timer_loop)

In [ ]:
# Cleanup
## Randomization may produce duplicates; we remove these with the following snippet:
get_unique_entries = np.unique(N_parts_Generated_by_Algo_2_history, return_index=True)[1]
N_parts_Generated_by_Algo_2_history_report = N_parts_Generated_by_Algo_2_history[get_unique_entries]

# Write
## Prediction Qualities
performance_Architope_history_report_MAE_train = (performance_Architope_history[0,0,:])[get_unique_entries]
performance_Architope_history_report_MAE_test = (performance_Architope_history[0,1,:])[get_unique_entries]
performance_Architope_history_report_MSE_train = (performance_Architope_history[1,0,:])[get_unique_entries]
performance_Architope_history_report_MSE_test = (performance_Architope_history[1,1,:])[get_unique_entries]
## Model Complexities
L_Times = (Architope_Model_Complexity_history[:,1].reshape(-1,))[get_unique_entries]
P_Times = (Architope_Model_Complexity_history[:,0].reshape(-1,))[get_unique_entries]
N_Params = (N_params_architope_hist.reshape(-1,))[get_unique_entries]
mean_subpattern_widths_hist = (height_mean_hist.reshape(-1,))[get_unique_entries]
AIC_Like = (Architope_Model_Complexity_history[:,3].reshape(-1,))[get_unique_entries]
Eff = (Architope_Model_Complexity_history[:,4].reshape(-1,))[get_unique_entries]
N_neurons_per_input = (N_neurons_per_input.reshape(-1,))#[get_unique_entries]
## Misc
Deep_Zero_Sets_timer_hist = Deep_Zero_Sets_timer_hist#[get_unique_entries]

# Record Benchmark Complexities
## PCNN-lgt
performance_lgt_ffNN_report_MAE_train = (performance_PCNN_ffNN_logistic_hist[0,0,:])[get_unique_entries]
performance_lgt_ffNN_report_MAE_test = (performance_PCNN_ffNN_logistic_hist[0,1,:])[get_unique_entries]
performance_lgt_ffNN_report_MSE_train = (performance_PCNN_ffNN_logistic_hist[1,0,:])[get_unique_entries]
performance_lgt_ffNN_report_MSE_test = (performance_PCNN_ffNN_logistic_hist[1,1,:])[get_unique_entries]
N_params_PCNN_logistic_hist = (N_params_subPatterns_hist[get_unique_entries]*N_parts_Generated_by_Algo_2_history_report) + N_parts_Generated_by_Algo_2_history_report
N_params_PCNN_logistic_hist_per_input = (N_params_subPatterns_hist[get_unique_entries] + N_parts_Generated_by_Algo_2_history_report)
P_time_PCNN_lgt = logistic_time_hist[get_unique_entries] + P_Times - Deep_Zero_Sets_timer_hist[get_unique_entries]
L_time_PCNN_lgt = logistic_time_hist[get_unique_entries] + L_Times - Deep_Zero_Sets_timer_hist[get_unique_entries]
## PCNN-bag
performance_PCNN_ffNN_bag_report_MAE_train = (performance_bagged_ffNN_hist[0,0,:])[get_unique_entries]
performance_PCNN_ffNN_bag_report_MAE_test = (performance_bagged_ffNN_hist[0,1,:])[get_unique_entries]
performance_PCNN_ffNN_bag_report_MSE_train = (performance_bagged_ffNN_hist[1,0,:])[get_unique_entries]
performance_PCNN_ffNN_bag_report_MSE_test = (performance_bagged_ffNN_hist[1,1,:])[get_unique_entries]
N_params_PCNN_ffNN_bag =  (N_neurons_per_input[get_unique_entries] - Deep_Zero_Sets_timer_hist[get_unique_entries])*N_parts_Generated_by_Algo_2_history_report
N_params_PCNN_ffNN_bag_per_input = N_params_PCNN_ffNN_bag
P_time_PCNN_bag = baggin_time_hist[get_unique_entries] + P_Times - Deep_Zero_Sets_timer_hist[get_unique_entries]
L_time_PCNN_bag = baggin_time_hist[get_unique_entries] + L_Times - Deep_Zero_Sets_timer_hist[get_unique_entries]

---
# Get Best PCNN
This is identified as the PCNN with the smallest training MAE.

---

In [ ]:
print('PCNN Historical Update (Training - MAE):')
print(performance_lgt_ffNN_report_MAE_train)
print('PCNN Historical Update (Testing - MAE):')
print(performance_lgt_ffNN_report_MAE_test)

In [ ]:
# best_N_parts = np.argmin(performance_Architope_history_report_MAE_test)
best_N_parts = np.maximum(1,np.argmin(performance_Architope_history_report_MAE_test))
PCNN_N_parts = N_parts_Generated_by_Algo_2_history[best_N_parts]
# Get PCNN Performance Metrics
PCNN_MAE_train = performance_Architope_history_report_MAE_train[best_N_parts]
PCNN_MAE_test = performance_Architope_history_report_MAE_test[best_N_parts]
PCNN_MSE_train = performance_Architope_history_report_MSE_train[best_N_parts]
PCNN_MSE_test = performance_Architope_history_report_MSE_test[best_N_parts]
PCNN_performance_all = performance_Architope_history[:,:,best_N_parts]
## Model Complexities
PCNN_L_time = L_Times[best_N_parts]
PCNN_P_time = P_Times[best_N_parts]
PCNN_subpattern_widths_hist = mean_subpattern_widths_hist[best_N_parts]
PCNN_AIC_Like = AIC_Like[best_N_parts]
PCNN_Eff = Eff[best_N_parts]
PCNN_N_neurons_per_input = N_neurons_per_input[best_N_parts]

---
# Get Other Benchmark(s)
## Feedforward Neural Network (ffNN) Benchmark
---

In [ ]:
print('Get Vanilla FFNN Model (i.e.: classical ffNN trained with ADAM)')
exec(open('Grid_Enhanced_Network.py').read())
if 'y_hat_ffNN_train' not in locals():
    param_grid_FFNNs['input_dim'] = [int(X_train.shape[1])]
    param_grid_FFNNs['ouput_dim'] = [int(y_train.shape[1])]
    ffNN_train_time = time.time()
    y_hat_ffNN_train, y_hat_ffNN_test, N_neurons_ffNN = build_ffNN(n_folds = 4, 
                                                                   n_jobs = n_jobs,
                                                                   n_iter = n_iter, 
                                                                   param_grid_in = param_grid_FFNNs, 
                                                                   X_train= X_train.to_numpy(), 
                                                                   y_train=y_train,
                                                                   X_test_partial=X_train.to_numpy(),
                                                                   X_test=X_test.to_numpy(),
                                                                   NOCV=True)
    #### Compute Performance
    # Compute Peformance
    performance_ffNN = reporter(y_train_hat_in=y_hat_ffNN_train,
                                y_test_hat_in=y_hat_ffNN_test,
                                y_train_in=y_train,
                                y_test_in=y_test)
    P_time_ffNN = time.time() - ffNN_train_time
    L_time_ffNN = P_time_ffNN
Width_ffNN = param_grid_Vanilla_Nets['height'][0]; Width_neurons_ffNN = Width_ffNN
MAE_ffNN = np.repeat(performance_ffNN.to_numpy()[0,1],len(N_parts_Generated_by_Algo_2_history_report))
MSE_ffNN = np.repeat(performance_ffNN.to_numpy()[1,1],len(N_parts_Generated_by_Algo_2_history_report))
N_neurons_ffNN_plot = np.repeat(N_neurons_ffNN,len(N_parts_Generated_by_Algo_2_history_report))#; N_neurons_ffNN = N_neurons_per_input_ffNN
N_neurons_per_input_ffNN = np.repeat(N_neurons_ffNN,len(N_parts_Generated_by_Algo_2_history_report))#; N_neurons_ffNN = N_neurons_per_input_ffNN
# Misc
ffNN_performance_all = performance_ffNN.to_numpy()
L_times_ffNN_plot = np.repeat(L_time_ffNN,len(N_parts_Generated_by_Algo_2_history_report))
P_times_ffNN_plot = np.repeat(P_time_ffNN,len(N_parts_Generated_by_Algo_2_history_report))
#----------------------------------#
print('Get Randomized FFNN Model')
#----------------------------------#
P_time_ffNN_rnd = P_Times[0]
L_time_ffNN_rnd = P_Times[0]
Width_ffNN_rnd = height_mean_hist[0]
# For: Plots
MAE_ffNN_rnd = np.repeat(performance_Architope_history_report_MAE_test[0],len(N_parts_Generated_by_Algo_2_history_report))
MSE_ffNN_rnd = np.repeat(performance_Architope_history_report_MSE_test[0],len(N_parts_Generated_by_Algo_2_history_report))
N_neurons_per_input_ffNN_rnd = np.repeat(N_neurons_per_input[0],len(N_parts_Generated_by_Algo_2_history_report))
Width_neurons_ffNN_rnd = np.repeat(mean_subpattern_widths_hist[0],len(N_parts_Generated_by_Algo_2_history_report))
N_neurons_ffNN_rnd = np.repeat(N_Params[0],len(N_parts_Generated_by_Algo_2_history_report))
# Misc
ffNN_performance_all_rnd = performance_Architope_history[:,:,0]
L_times_ffNN_rnd_plot = np.repeat(L_time_ffNN_rnd,len(N_parts_Generated_by_Algo_2_history_report))
P_times_ffNN_rnd_plot = np.repeat(P_time_ffNN_rnd,len(N_parts_Generated_by_Algo_2_history_report))

performance_ffNN = reporter(y_train_hat_in=y_hat_ffNN_train,
                            y_test_hat_in=y_hat_ffNN_test,
                            y_train_in=y_train,
                            y_test_in=y_test)

## Gradient Boosted Random Forest Regressor (GBRF)

In [ ]:
# Update User #
#-------------#
print('Training Gradient-Boosted Random Forest: In-progress...')
# Run from External Script
print('Get Vanilla FFNN Model (i.e.: classical ffNN trained with ADAM)')
exec(open('Grid_Enhanced_Network.py').read())
if 'y_train_hat_random_forest_Gradient_boosting' not in locals():
    exec(open('Gradient_Boosted_Random_Forest_Regressor.py').read())
    print('Best GBRF Estimator:')
    print(random_forest_trained)
# Update User #
#-------------#
print('Training of Gradient-Boosted Random Forest: Complete!')

# Compute N Trainable Parameters

In [ ]:
# Compute Each Constituent's Number of Trainable Parameters
N_PCNN_trainable_parameters_per_subpatter_total = param_grid_Vanilla_Nets['height'][0]*param_grid_Vanilla_Nets['output_dim'][0]
N_PCNN_trainable_parameters_subpatters_total = PCNN_N_parts *N_PCNN_trainable_parameters_per_subpatter_total
N_PCNN_trainable_deep_zero_sets = PCNN_N_parts *param_grid_Vanilla_Nets['height'][0]
# For Benchmark(s)
N_PCNN_trainable_logistic_deep_classifier = PCNN_N_parts*param_grid_Vanilla_Nets['input_dim'][0]
N_trainable_FFNN = (param_grid_FFNNs['height'][0]**2)*param_grid_FFNNs['depth'][0] +param_grid_FFNNs['output_dim'][0]*param_grid_FFNNs['height'][0] + param_grid_FFNNs['input_dim'][0]*param_grid_FFNNs['height'][0]

# Compute N Paramters
N_PCNN_trainable_params = N_PCNN_trainable_parameters_subpatters_total + N_PCNN_trainable_deep_zero_sets
N_ffNN_bag_trainable_params = N_PCNN_trainable_parameters_subpatters_total
N_ffNN_lgt_trainable_params = N_PCNN_trainable_parameters_subpatters_total + N_PCNN_trainable_logistic_deep_classifier
N_ffNN_rnd = param_grid_Vanilla_Nets['height'][0]*param_grid_Vanilla_Nets['output_dim'][0]

# Write Tables

### Prediction Metric(s)
#### Write Predictive Performance Dataframe(s)

In [ ]:
for jj in range(2):
    if jj == 0:
        Relative_MAE_to_FFNN = False
    else:
        Relative_MAE_to_FFNN = True
    print(Relative_MAE_to_FFNN)
    #### Generate Table ####
    #----------------------#
    # Extract MAEs from Test-Set
    if Relative_MAE_to_FFNN == False:
        MAE_test_time_table = np.array([performance_ffNN.test.MAE,
                                    MSE_ffNN_rnd[0],
                                    performance_PCNN_ffNN_bag_report_MAE_test[best_N_parts],
                                    performance_lgt_ffNN_report_MAE_test[best_N_parts],
                                    Gradient_boosted_tree.test.MAE,
                                    PCNN_MAE_test])
        ## In-Line Time
        L_time_table = np.round(np.array([L_time_ffNN,
                                 L_time_ffNN_rnd,
                                 L_time_PCNN_bag[best_N_parts],
                                 L_time_PCNN_lgt[best_N_parts],
                                 Gradient_boosted_Random_forest_time,
                                 PCNN_L_time]),4)
        ## Extract Parallelized Training Times
        P_time_table = np.array(['-',
                                 '-',
                                 np.round(P_time_PCNN_bag[best_N_parts],4),
                                 np.round(P_time_PCNN_lgt[best_N_parts],4),
                                 '-',
                                 np.round(PCNN_P_time,4)])
        ## Extract Total Number of Active Neurons per input
        N_params_table_per_intput = np.array([N_trainable_FFNN,
                                                       N_trainable_FFNN,
                                                       N_params_PCNN_ffNN_bag_per_input[best_N_parts],
                                                       N_params_PCNN_logistic_hist_per_input[best_N_parts],
                                                       N_tot_params_in_forest,
                                                       N_Params[best_N_parts]])
        ## Extract Total Number of Active Neurons
#         N_params_table = np.round(np.array([N_neurons_ffNN,
#                                             N_neurons_ffNN,
#                                             N_params_PCNN_ffNN_bag[best_N_parts],
#                                             N_params_PCNN_logistic_hist[best_N_parts],
#                                             N_tot_params_in_forest,
#                                             PCNN_N_neurons_per_input]),0)   
        ## Extract Total Number of Trainable Parameters
        N_params_table = np.array([N_trainable_FFNN,
                                            N_ffNN_rnd,
                                            N_ffNN_bag_trainable_params,
                                            N_ffNN_lgt_trainable_params,
                                            N_tot_params_in_forest,
                                            N_PCNN_trainable_params])    
        # N Parts
        N_parts= np.array([1,
                            1,
                            PCNN_N_parts,
                            PCNN_N_parts,
                            1,
                            PCNN_N_parts])

    else:
        MAE_test_time_table = np.array([1,
                                        MSE_ffNN_rnd[0]/performance_ffNN.test.MAE,
                                        performance_PCNN_ffNN_bag_report_MAE_test[best_N_parts]/performance_ffNN.test.MAE,
                                        performance_lgt_ffNN_report_MAE_test[best_N_parts]/performance_ffNN.test.MAE,
                                        Gradient_boosted_tree.test.MAE/performance_ffNN.test.MAE,
                                        PCNN_MAE_test/performance_ffNN.test.MAE])
        ## In-Line Time
        L_time_table = np.array([1,
                                 L_time_ffNN_rnd/L_time_ffNN,
                                 L_time_PCNN_bag[best_N_parts]/L_time_ffNN,
                                 L_time_PCNN_lgt[best_N_parts]/L_time_ffNN,
                                 Gradient_boosted_Random_forest_time/L_time_ffNN,
                                 PCNN_L_time/L_time_ffNN])
        ## Extract Parallelized Training Times
        P_time_table = np.array(['-',
                                 '-',
                                 P_time_PCNN_bag[best_N_parts]/L_time_ffNN,
                                 P_time_PCNN_lgt[best_N_parts]/L_time_ffNN,
                                 '-',
                                 PCNN_P_time/L_time_ffNN])
        ## Extract Total Number of Active Neurons per input
        N_params_table_per_intput = np.round(np.array([1,
                                                       1,
                                                       N_params_PCNN_ffNN_bag_per_input[best_N_parts]/N_trainable_FFNN,
                                                       N_params_PCNN_logistic_hist_per_input[best_N_parts]/N_trainable_FFNN,
                                                       N_tot_params_in_forest/N_trainable_FFNN,
                                                       N_Params[best_N_parts]/N_trainable_FFNN]))
        ## Extract Total Number of Active Neurons
#         N_params_table = np.round(np.array([1,
#                                             1,
#                                             N_params_PCNN_ffNN_bag[best_N_parts]/N_neurons_ffNN,
#                                             N_params_PCNN_logistic_hist[best_N_parts]/N_neurons_ffNN,
#                                             N_tot_params_in_forest/N_neurons_ffNN,
#                                             PCNN_N_neurons_per_input/N_neurons_ffNN]),0)
        N_params_table = np.array([1,
                                            N_ffNN_rnd/N_trainable_FFNN,
                                            N_ffNN_bag_trainable_params/N_trainable_FFNN,
                                            N_ffNN_lgt_trainable_params/N_trainable_FFNN,
                                            N_tot_params_in_forest/N_trainable_FFNN,
                                            N_PCNN_trainable_params/N_trainable_FFNN])
        # N Parts
        N_parts= np.array([1,
                            1,
                            PCNN_N_parts,
                            PCNN_N_parts,
                            1,
                            PCNN_N_parts])

    #### Write Main Table for Paper ####
    #----------------------------------#
    Table_Final = pd.DataFrame({'MAE': MAE_test_time_table,
                                'L. Time': L_time_table,
                                'P. Time': P_time_table,
                                'N. Train. Par': N_params_table,
#                                 'N. Par/x': N_params_table_per_intput,
                                'N. Parts': N_parts},index=['FFNN','FFNN-RND','FFNN-BAG','FFNN-LGT','GBRF','PCNN'])
    # Add Further Information for Synthetic Experiments
    if Option_Function == 'Sythetic':
        # Write Experiment Facts
        Experiment_Dimension = pd.DataFrame({'d': np.repeat(D_in,Table_Final.shape[0])},index=Table_Final.index)
        Variance_Experiment = pd.DataFrame({'$\sigma$': np.repeat(noise_level,Table_Final.shape[0])},index=Table_Final.index)
        N_Samples_Experiment = pd.DataFrame({'N': np.repeat(N,Table_Final.shape[0])},index=Table_Final.index)
        Tailedness_Experiment = pd.DataFrame({'$\nu$': np.repeat(tailedness,Table_Final.shape[0])},index=Table_Final.index)
        Partitioning_Frequency_Experiment = pd.DataFrame({'r': np.repeat(frequency_or_self_paritioning,Table_Final.shape[0])},index=Table_Final.index)
        

        # Append To Final Table
        Table_Final = Table_Final.join(Experiment_Dimension)
        Table_Final = Table_Final.join(Variance_Experiment)
        Table_Final = Table_Final.join(N_Samples_Experiment)
        Table_Final = Table_Final.join(Tailedness_Experiment)
        Table_Final = Table_Final.join(Partitioning_Frequency_Experiment)
    
    # Write to File
    Table_Final.to_latex(('outputs/tables/'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'ReltoFFNN'+str(Relative_MAE_to_FFNN)+'Table.tex'),
                         caption = 'Prediction and Complexity Metrics: '+str(Option_Function)+'Relative to FFNN'+str(Relative_MAE_to_FFNN),
                         label='tab__'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q),
                         float_format="{:0.2e}".format)
    #--------------======---------------#
    print('Display Performance Metrics')
    #--------------======---------------#
    for j in range(3):
        print('-----------------------------------')
    print('Final Results of This Experiment: ')
    print('Relative to FFNN: '+str(Relative_MAE_to_FFNN))
    print(Table_Final)
    for j in range(3):
        print('-----------------------------------')

# Reports Table for Live User
Table_Final

---
## Plots
---

In [ ]:
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
plt.savefig('./outputs/plotsANDfigures/dump.eps')
plt.show()

## MSE

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("MSE")
plt.xlabel("N. Parts")
plt.ylabel("MSE")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         performance_Architope_history_report_MSE_test,
         label = 'PCNN',
         color='seagreen',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         MSE_ffNN,
         label = 'FFNN',
         color='darkmagenta',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         MSE_ffNN_rnd,
         label = 'FFNNN-RND',
         color='pink',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         performance_lgt_ffNN_report_MSE_test,
         label = 'ffNN-LGT',
         color='red',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         performance_PCNN_ffNN_bag_report_MSE_test,
         label = 'ffNN-BAG',
         color='orange',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
# Add Legend
plt.legend(loc="upper left")

# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_MSE_test___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
plt.show()

## MAE

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("MAE")
plt.xlabel("N. Parts")
plt.ylabel("MAE")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         performance_Architope_history_report_MAE_test,
         label = 'PCNN',
         color='seagreen',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         MAE_ffNN,
         label = 'FFNN',
         color='darkmagenta',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         MAE_ffNN_rnd,
         label = 'FFNNN-RND',
         color='pink',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         performance_lgt_ffNN_report_MAE_test,
         label = 'FFNN-LGT',
         color='red',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
# plt.plot(N_parts_Generated_by_Algo_2_history_report,
#          performance_PCNN_ffNN_bag_report_MAE_test,
#          label = 'FFNN-BAG',
#          color='orange',
#          linewidth=2.5,
#          linestyle = '-.', 
#          alpha = .75)

# Export #
#--------#
# SAVE Figure to .eps
plt.legend(loc="upper left")
plt.savefig('./outputs/plotsANDfigures/Ablation_MAE___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
plt.show()

# P. Time

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("P. Time")
plt.xlabel("N. Parts")
plt.ylabel("P. Time")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         L_Times,
         label = 'PCNN',
         color='seagreen',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         L_times_ffNN_plot,
         label = 'FFNN',
         color='darkmagenta',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         L_times_ffNN_rnd_plot,
         label = 'FFNNN-RND',
         color='pink',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         L_time_PCNN_lgt,
         label = 'FFNN-LGT',
         color='red',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         L_time_PCNN_bag,
         label = 'FFNN-BAG',
         color='orange',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.legend(loc="upper left")


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_L_Time___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
plt.show()

# L. Times

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("L. Time")
plt.xlabel("N. Parts")
plt.ylabel("L. Time")

# Generate Plots #
#----------------#
# Plot Signal
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         P_Times,
         label = 'PCNN',
         color='seagreen',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         P_times_ffNN_plot,
         label = 'FFNN',
         color='darkmagenta',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         P_times_ffNN_rnd_plot,
         label = 'FFNNN-RND',
         color='pink',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         P_time_PCNN_lgt,
         label = 'FFNN-LGT',
         color='red',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         P_time_PCNN_bag,
         label = 'FFNN-BAG',
         color='orange',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.legend(loc="upper left")

# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_P_Time___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
plt.show()

## N-Params

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("N. Params")
plt.xlabel("N. Parts")
plt.ylabel("N. Params")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_Params,
         label = 'PCNN',
         color='seagreen',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_neurons_ffNN_plot,
         label = 'FFNN',
         color='darkmagenta',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_neurons_ffNN_rnd,
         label = 'FFNNN-RND',
         color='pink',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_params_PCNN_logistic_hist,
         label = 'FFNN-LGT',
         color='red',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_params_PCNN_ffNN_bag,
         label = 'FFNN-BAG',
         color='orange',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.legend(loc="upper left")


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_N_Params___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
plt.show()

# Number of Active Neurons Per Input

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("Active Neurons per. Input")
plt.xlabel("N. Parts")
plt.ylabel("Active Neurons per. Input")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_neurons_per_input,
         label = 'PCNN',
         color='seagreen',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_neurons_ffNN_plot,
         label = 'FFNN',
         color='darkmagenta',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_neurons_ffNN_rnd,
         label = 'FFNNN-RND',
         color='pink',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_params_PCNN_logistic_hist_per_input,
         label = 'FFNN-LGT',
         color='red',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_params_PCNN_ffNN_bag_per_input,
         label = 'FFNN-BAG',
         color='orange',
         linewidth=2.5,
         linestyle = '-.', 
         alpha = .75)
plt.legend(loc="upper left")


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_Active_Neurons_per_input___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
plt.show()

# Mean Widths for Sub-Pattern Networks

In [ ]:
# sns.set()
# # Initialize Plot #
# #-----------------#
# plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# # Format Plot #
# #-------------#
# plt.title("Mean Subpattern Widths")
# plt.xlabel("N. Parts")
# plt.ylabel("Mean Subpattern Widths")

# # Generate Plots #
# #----------------#
# # Plot Signal
# # plt.plot(N_parts_Generated_by_Algo_2_history_report,
# #          mean_subpattern_widths_hist,
# #          label = 'ffNN-lgt',
# #          color='red',
# #          linewidth=2.5,
# #          linestyle = '-.', 
# #          alpha = .75)
# # plt.plot(N_parts_Generated_by_Algo_2_history_report,
# #          mean_subpattern_widths_hist,
# #          label = 'ffNN-bag',
# #          color='orange',
# #          linewidth=2.5,
# #          linestyle = '-.', 
# #          alpha = .75)
# plt.plot(N_parts_Generated_by_Algo_2_history_report,
#          mean_subpattern_widths_hist,
#          label = 'PCNN',
#          color='seagreen',
#          linewidth=2.5)
# plt.plot(N_parts_Generated_by_Algo_2_history_report,
#          mean_subpattern_widths_hist,
#          label = 'ffNN',
#          color='darkmagenta',
#          linewidth=2.5)
# plt.legend(loc="upper left")


# # Export #
# #--------#
# # SAVE Figure to .eps
# plt.savefig('./outputs/plotsANDfigures/Ablation_Mean_Widths___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
# plt.show()

---

# MAE Progression (PCNN only)

---

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("MAE")
plt.xlabel("N. Parts")
plt.ylabel("MAE")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         performance_Architope_history_report_MAE_test,
         label = 'PCNN',
         color='seagreen',
         linewidth=2.5)
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         MAE_ffNN,
         label = 'FFNN',
         color='darkmagenta',
         linewidth=2.5)

# Export #
#--------#
# SAVE Figure to .eps
plt.legend(loc="upper left")
plt.savefig('./outputs/plotsANDfigures/Ablation_MAE___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.pdf')
plt.show()

# Table(s)

In [ ]:
print('😃😃 Done!!! 😃😃')
print('Best Number of Part(s): ', N_parts_Generated_by_Algo_2_history[best_N_parts])
for j in range(3):
    print('---------------------')
print('Prediction Metric(s)')
for j in range(3):
    print('---------------------')
#--------------======---------------#
print('Display Performance Metrics')
#--------------======---------------#
print(Table_Final.round(3))
for j in range(3):
    print('---------------------')
print('Prediction Metric(s)')
for j in range(3):
    print('---------------------')
print('😃😃 Have a great day!! 😃😃 ')

---

---

---

# Fin

---

---

---